In [ ]:
import yaml

CREDS = yaml.load(open("credentials.yml"), Loader=yaml.FullLoader)


print(CREDS["AKASH_KEY"])

sk-gjL7We9IbD2xVKd6Od9ozA


In [24]:
import requests

def get_dashboard_data():
    url = "https://console-api.akash.network/v1/dashboard-data"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching dashboard data: {e}")
        return None


dashboard_data = get_dashboard_data()
print(dashboard_data)


{'chainStats': {'bondedTokens': 114867961387663, 'totalSupply': 264868746067143, 'communityPool': 9841680343813.512, 'inflation': 0.13, 'stakingAPR': 0.1798566105261797, 'height': 20253718, 'transactionCount': 21764343}, 'now': {'date': '2025-02-15T20:41:43.633Z', 'height': 20253715, 'activeLeaseCount': 936, 'totalLeaseCount': 297935, 'dailyLeaseCount': 604, 'totalUAktSpent': 699366459205.223, 'dailyUAktSpent': 4332459225.015869, 'totalUUsdcSpent': 513590540768.9159, 'dailyUUsdcSpent': 5056505354.349365, 'totalUUsdSpent': 2151304475610.726, 'dailyUUsdSpent': 13391405434.865234, 'activeCPU': 5010470, 'activeGPU': 627, 'activeMemory': 21617414581248, 'activeStorage': 65266395165056}, 'compare': {'date': '2025-02-14T20:41:43.975Z', 'height': 20239420, 'activeLeaseCount': 965, 'totalLeaseCount': 297331, 'dailyLeaseCount': 681, 'totalUAktSpent': 695033999980.2072, 'dailyUAktSpent': 4219047137.273926, 'totalUUsdcSpent': 508534035414.5665, 'dailyUUsdcSpent': 5339369764.453674, 'totalUUsdSpent

In [25]:
apr =dashboard_data["chainStats"]["stakingAPR"]

total_leases = dashboard_data["now"]["totalLeaseCount"]
daily_leases = dashboard_data["now"]["dailyLeaseCount"]

total_usd_spent = dashboard_data["now"]["totalUUsdSpent"]
daily_usd_spent = dashboard_data["now"]["dailyUUsdSpent"]

# print(f"{total_usd_spent:,.0f}")
# print(f"{daily_usd_spent:,.0f}")

total_usd_spent

2151304475610.726

Agent stuff

In [ ]:
from httpx import AsyncClient
import sys
sys.path.insert(0, r'C:\Users\Ale\Documents\code\pythonista\M10 server\infra\dc_version')
from shared.API_client import get_query
from pydantic_ai import Agent, RunContext
from openai import AsyncOpenAI
from pydantic_ai.models.openai import OpenAIModel
from shared.glenda import agent_briefing as ab


client = AsyncOpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama'
)

model = OpenAIModel("qwen2.5:7b", openai_client=client)


db_read_agent = Agent(
    model=model, 
    system_prompt="You are an SQL expert with very good understanding of database design expressed through python code and human writing",
)


@db_read_agent.tool
async def read_db(ctx: RunContext[None], sqlquery: str ) -> str:
      
      """ 
        Pass SQL query as argument to the get_query() method.
        Present findings. You can use dependency if you wish but it is not necessary.
        Search for keywords that match column names to infer which table to search.
        Always write the table name with capital letter and always check for a column name that
        matches the column name you want to query.Sometimes number can be abbreviated as nr, 
        and personal number can be pers nr, but this means column personal_number.
        Always interpret pers nr as personal_number column.
        
        Args:
            ctx: The context.
            sqlquery: The query to search through the database.

        Return:
            str: The search results as a formatted string.
      """
      
      

      print("--->>>",sqlquery)
      results = get_query(sqlquery)
      
      return results if results else "No results for query."


async def main():
    async with AsyncClient() as client:

         result = await db_read_agent.run("Explain the M10.db database schema")


         return result.data


# Replace asyncio.run(main()) with:

res = await main()

res



In [ ]:
import asyncio
from httpx import AsyncClient
import sys

from shared.API_client import get_query
from pydantic_ai import Agent, RunContext

 

db_read_agent = Agent(
    model="ollama:qwen2.5:7b", 
    result_type=str,
    system_prompt="You are an SQL expert with very good understanding of database design expressed through python code and human writing",
    deps_type=None,   # no dependencies yet
    retries=2
)


@db_read_agent.tool
async def read_db(ctx: RunContext[None], sqlquery: str ) -> str:
      """ 
        Search through sqlite database M10.db by translating given command into SQL query.
        Pass SQL query as argument to the get_query() method.
        Present findings. You can use dependency if you wish but it is not necessary.
        Search for keywords that match column names to infer which table to search.
        For example a prompt containing "contacts" and "channel" should generate a query
        for Contacts table and contact_channel column. If returned as a dictionary reformat it to string.
        Always write the table name with capital letter and always check for a column name that
        matches the column name you want to query.
        
        Args:
            ctx: The context.
            sqlquery: The query to search through the database.

        Return:
            str: The search results as a formatted string.
        """

      print("Agent generated SQL command: ", sqlquery)
      results = get_query(sqlquery)
      
      return results if results else "No results for query."


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 


async def main():
    async with AsyncClient() as client:
        result = await db_read_agent.run("Get f_name and emails of all members who's p_nr starts with 05")
        
        print(result.data)
        return result.data

asyncio.run(main())

# Replace asyncio.run(main()) with:
#loop = asyncio.get_event_loop()
#res = await loop.create_task(main())

#res


In [ ]:

    def glenda_agent(self, prompt):

        model = OpenAIModel("qwen2.5:7b", openai_client=self.local_client)


        db_read_agent = Agent(
            model=model, 
            result_type=str,
            system_prompt="You are an SQL expert with very good understanding of database design expressed through python code and human writing",
            deps_type=None,   # no dependencies yet
            retries=2
        )


        @db_read_agent.tool
        async def read_db(ctx: RunContext[None], sqlquery: str ) -> str:

            ab.read_db_instructions
            ab.query_examples
            ab.database_schema

            print("--->>>",sqlquery)
            results = get_query(sqlquery)
            
            return results if results else "No results for query."


        async def main():
            async with AsyncClient() as client:

                result = await db_read_agent.run_stream("Get f_name and emails of all members who's p_nr starts with 05")


                return result.data
